In [130]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [131]:
%cd /content/drive/MyDrive/secom

/content/drive/MyDrive/secom


In [132]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from scipy.stats import randint
from xgboost import XGBClassifier
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split


In [133]:
df = pd.read_csv("secom.data", delim_whitespace=True, header=None)

df.to_csv("secom.csv", index=False)


<ipython-input-133-0c31339a0ac0>:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv("secom.data", delim_whitespace=True, header=None)


In [134]:
df_labels = pd.read_csv("secom_labels.data", delim_whitespace=True, header=None)

df.to_csv("secom_labels.csv", index=False)


<ipython-input-134-05a50dee3534>:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_labels = pd.read_csv("secom_labels.data", delim_whitespace=True, header=None)


In [136]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432


In [137]:
df_labels.columns = ["label", "timestamp"]


In [138]:
df_labels.head()

,label,timestamp
0,-1,19/07/2008 11:55:00
1,-1,19/07/2008 12:32:00
2,1,19/07/2008 13:17:00
3,-1,19/07/2008 14:43:00
4,-1,19/07/2008 15:22:00


In [139]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.0044,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432


In [140]:
# Assign column names to features
df.columns = [f"feature_{i}" for i in range(df.shape[1])]

In [141]:
# Combine data and labels
df_combined = pd.concat([df, df_labels], axis=1)
df_combined.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_582,feature_583,feature_584,feature_585,feature_586,feature_587,feature_588,feature_589,label,timestamp
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1,19/07/2008 11:55:00
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1,19/07/2008 12:32:00
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1,19/07/2008 13:17:00
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1,19/07/2008 14:43:00
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1,19/07/2008 15:22:00


In [142]:
# Map labels to binary values
df_combined["label"] = df_combined["label"].map({-1: 0, 1: 1})

In [143]:
# Handle missing data
threshold = 0.4
# Drop columns with more than 40% missing values
df_combined = df_combined.dropna(thresh=int((1 - threshold) * len(df_combined)), axis=1)

In [144]:
# Drop timestamp column (not useful for modeling)
df_combined = df_combined.drop(columns=["timestamp"], errors="ignore")

In [145]:
# Fill remaining missing values with column means
for col in df_combined.select_dtypes(include=[np.number]).columns:
    df_combined[col] = df_combined[col].fillna(df_combined[col].mean())

In [147]:
df_combined.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_577,feature_582,feature_583,feature_584,feature_585,feature_586,feature_587,feature_588,feature_589,label
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,14.9509,0.5005,0.0118,0.0035,2.3630,0.021458,0.016475,0.005283,99.670066,0
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,10.9003,0.5019,0.0223,0.0055,4.4447,0.009600,0.020100,0.006000,208.204500,0
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,9.2721,0.4958,0.0157,0.0039,3.1745,0.058400,0.048400,0.014800,82.860200,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,8.5831,0.4990,0.0103,0.0025,2.0544,0.020200,0.014900,0.004400,73.843200,0
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,10.9698,0.4800,0.4766,0.1045,99.3032,0.020200,0.014900,0.004400,73.843200,0


In [165]:
# Prepare data for modeling
X = df_combined.values
y = df_combined["label"].values

In [166]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Shapes:")
print("  X_train:", X_train.shape, " y_train:", y_train.shape)
print("  X_test: ", X_test.shape, " y_test: ", y_test.shape)

Shapes:
  X_train: (1253, 559)  y_train: (1253,)
  X_test:  (314, 559)  y_test:  (314,)


In [167]:
# Define pipeline with PCA and RandomForestClassifier
pipeline_pca = ImbPipeline([
    ("scaler", StandardScaler()),
    ("oversampler", SMOTE(random_state=42)),
    ("pca", PCA()),
    ("clf", RandomForestClassifier(random_state=42))
])

In [168]:
oversamplers = [
    SMOTE(random_state=42),
    BorderlineSMOTE(random_state=42),
    SMOTEENN(random_state=42),
    SMOTETomek(random_state=42)
]

In [169]:
# Define parameter grid
param_dist = {
    "oversampler": oversamplers,
    "pca__n_components": [0.90, 0.95, 10, 20, 40],
    "clf__n_estimators": randint(50, 301),
    "clf__max_depth": [None, 10, 20],
    "clf__max_features": ["sqrt", "log2", None],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 5],
    "clf__class_weight": [
        None,
        "balanced_subsample",
        {0: 1, 1: 2},
        {0: 1, 1: 3}
    ]
}

In [170]:
cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=2,
    random_state=42
)
scorer = make_scorer(balanced_accuracy_score)
random_search = RandomizedSearchCV(
    estimator=pipeline_pca,
    param_distributions=param_dist,
    n_iter=30,
    scoring=scorer,
    cv=cv,
    random_state=42,
    verbose=1,
    n_jobs=-1
)

random_search.fit(X_train, y_train)

print("Best Params:", random_search.best_params_)
print("Best CV Balanced Accuracy:", random_search.best_score_)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTEENN or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.valida

Best Params: {'clf__class_weight': {0: 1, 1: 2}, 'clf__max_depth': 20, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 10, 'clf__n_estimators': 236, 'oversampler': SMOTEENN(random_state=42), 'pca__n_components': 20}
Best CV Balanced Accuracy: 0.7684766214177978


In [171]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
test_bal_acc = balanced_accuracy_score(y_test, y_pred)
print("Test Balanced Accuracy:", test_bal_acc)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)
report = classification_report(y_test, y_pred, target_names=["pass", "fail"])
print(report)

Test Balanced Accuracy: 0.7478465789045994
Confusion Matrix:
 [[215  78]
 [  5  16]]
              precision    recall  f1-score   support

        pass       0.98      0.73      0.84       293
        fail       0.17      0.76      0.28        21

    accuracy                           0.74       314
   macro avg       0.57      0.75      0.56       314
weighted avg       0.92      0.74      0.80       314



In [172]:
# Evaluate on test data
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

print("Test Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["pass", "fail"]))

Test Balanced Accuracy: 0.7478465789045994
Confusion Matrix:
 [[215  78]
 [  5  16]]
              precision    recall  f1-score   support

        pass       0.98      0.73      0.84       293
        fail       0.17      0.76      0.28        21

    accuracy                           0.74       314
   macro avg       0.57      0.75      0.56       314
weighted avg       0.92      0.74      0.80       314



In [173]:
# Custom threshold evaluation
y_proba = best_model.predict_proba(X_test)[:, 1]
threshold = 0.4
y_pred_custom = (y_proba >= threshold).astype(int)

print("Confusion Matrix (Threshold=0.4):\n", confusion_matrix(y_test, y_pred_custom))
print(classification_report(y_test, y_pred_custom, target_names=["pass", "fail"]))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_custom))

Confusion Matrix (Threshold=0.4):
 [[167 126]
 [  3  18]]
              precision    recall  f1-score   support

        pass       0.98      0.57      0.72       293
        fail       0.12      0.86      0.22        21

    accuracy                           0.59       314
   macro avg       0.55      0.71      0.47       314
weighted avg       0.93      0.59      0.69       314

Balanced Accuracy: 0.7135543637250121


## Further Enhancement

In [174]:
pipeline_xgb = ImbPipeline([
    ("scaler", StandardScaler()),
    ("smote", SMOTE(random_state=42)),
    ("pca", PCA()),
    ("xgb", XGBClassifier(
        eval_metric='logloss',
        random_state=42
    ))
])


In [175]:
param_dist = {

    "pca__n_components": [0.90, 0.95, 10, 20, 40],
    "xgb__n_estimators": randint(50, 301),
    "xgb__max_depth": [3, 5, 7, 10],
    "xgb__learning_rate": uniform(0.01, 0.2),
    "xgb__subsample": uniform(0.5, 0.5),
    "xgb__colsample_bytree": uniform(0.5, 0.5),
    "xgb__scale_pos_weight": [1, 2, 3, 5, 10]
}


In [176]:

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
scorer = make_scorer(balanced_accuracy_score)

random_search = RandomizedSearchCV(
    estimator=pipeline_xgb,
    param_distributions=param_dist,
    n_iter=30,
    scoring=scorer,
    cv=cv,
    random_state=42,
    verbose=1,
    n_jobs=-1)

In [177]:
random_search.fit(X_train, y_train)

print("Best Params:", random_search.best_params_)
print("Best CV Balanced Accuracy:", random_search.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Fitting 10 folds for each of 30 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Best Params: {'pca__n_components': 40, 'xgb__colsample_bytree': 0.6008596011676981, 'xgb__learning_rate': 0.18915271913470388, 'xgb__max_depth': 3, 'xgb__n_estimators': 101, 'xgb__scale_pos_weight': 10, 'xgb__subsample': 0.7542853455823514}
Best CV Balanced Accuracy: 0.8483722976370036


In [178]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

test_bal_acc = balanced_accuracy_score(y_test, y_pred)
print("Test Balanced Accuracy:", test_bal_acc)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

report = classification_report(y_test, y_pred, target_names=["pass", "fail"])
print(report)


Test Balanced Accuracy: 0.8638875345359986
Confusion Matrix:
 [[283  10]
 [  5  16]]
              precision    recall  f1-score   support

        pass       0.98      0.97      0.97       293
        fail       0.62      0.76      0.68        21

    accuracy                           0.95       314
   macro avg       0.80      0.86      0.83       314
weighted avg       0.96      0.95      0.95       314



In [179]:
y_proba = best_model.predict_proba(X_test)[:, 1]
threshold = 0.4
y_pred_custom = (y_proba >= threshold).astype(int)

from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score

cm_custom = confusion_matrix(y_test, y_pred_custom)
print(cm_custom)
print(classification_report(y_test, y_pred_custom, target_names=["pass", "fail"]))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_custom))


[[282  11]
 [  5  16]]
              precision    recall  f1-score   support

        pass       0.98      0.96      0.97       293
        fail       0.59      0.76      0.67        21

    accuracy                           0.95       314
   macro avg       0.79      0.86      0.82       314
weighted avg       0.96      0.95      0.95       314

Balanced Accuracy: 0.8621810498943605
